Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [12]:
NAME = ""
COLLABORATORS = ""

---

In [13]:
import sys

import numpy as np
import unittest
import time

import collections
import pickle
import io

In [14]:
class KNearestNeighbor:
    """ a kNN classifier with L2 distance """
    def __init__(self):
        pass

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X, k=1, num_loops=0):
        if num_loops == 0:
            dists = self.compute_distances_no_loops(X)
        elif num_loops == 1:
            dists = self.compute_distances_one_loop(X)
        elif num_loops == 2:
            dists = self.compute_distances_two_loops(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)

        return self.predict_labels(dists, k=k)

    def compute_distances_two_loops(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
                
        for i in range(num_test):
            for j in range(num_train):
                # 1. вычисляем разницу между векторами (D-размерности)
                diff = X[i, :] - self.X_train[j, :]
                
                # 2. вычисляем квадрат L2 расстояния 
                squared_distance = np.sum(diff**2)
                
                # 3. берем квадратный корень (L2)
                dists[i, j] = np.sqrt(squared_distance)                             
        return dists

    def compute_distances_one_loop(self, X):

        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        for i in range(num_test):            
            # 1 --- выбираем тестовую строку (1, D)
            test_point = X[i, :]
            
            # 2 --- вычисляем разницу
            diff = self.X_train - test_point
            sq_diff = np.sum(diff**2, axis=1)
            
            # 3 --- записываем
            dists[i, :] = np.sqrt(sq_diff)

        return dists

    def compute_distances_no_loops(self, X):
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
                
        # 1 --- вычисляем квадрат L2 нормы для каждой точки 
        sum_sq_test = np.sum(X**2, axis=1, keepdims=True)
        sum_sq_train = np.sum(self.X_train ** 2, axis=1, keepdims=True)
        
        # 2 --- вычисляем -2 * скаляр произ (-2 * A^T B)
        dot_product = -2 * np.dot(X, self.X_train.T)                
                
        # 3 --- считаем расстояние
        dists = sum_sq_test + sum_sq_train - dot_product  
        
        return dists

    def predict_labels(self, dists, k=1):
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):

            
            # 1 --- находим индексы k ближайших соседей для i-й тестовой точки.
            closest_indices = np.argsort(dists[i, :])[:k]
            
            # 2 --- используем self.y_train для получения меток по найденным индексам.
            closest_y = self.y_train[closest_indices] 

            from collections import Counter
            
            # 3 --- подсчет частоты меток
            vote_counts = Counter(closest_y)
            
            # 4 --- получаем метки с максимальным голосом (может быть несколько при ничьей)
            max_count = vote_counts.most_common(1)[0][1]
            winning_labels = [label for label, count in vote_counts.items() if count == max_count]
            
            # 5 --- азрешение ничьей: выбираем меньшую метку
            y_pred[i] = min(winning_labels) 


        return y_pred


### Test 1: Two loops (0.05 points)

In [15]:
# do not change this cell
knn_test = KNearestNeighbor()



In [16]:
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    tic = time.time()
    output = f(*args)
    toc = time.time()
    return toc - tic, output

knn_test.fit(X_train, y_train)

np.random.seed(42)
two_loop_time, out_2_loops = time_function(knn_test.compute_distances_two_loops, X_test)
assert np.allclose(real_distances, out_2_loops, atol=1e-6)

NameError: name 'X_train' is not defined

### Test 2: One loop (0.1 points)

In [ ]:
np.random.seed(42)
one_loop_time, out_1_loops = time_function(knn_test.compute_distances_one_loop, X_test)
assert np.allclose(real_distances, out_1_loops, atol=1e-6)

### Test 3: No loops (0.4 points)

In [ ]:
np.random.seed(42)
no_loop_time, out_no_loops = time_function(knn_test.compute_distances_no_loops, X_test)
assert np.allclose(real_distances, out_no_loops, atol=1e-6)


### Test 4: No loops timing (0.15 points)

In [ ]:
np.random.seed(42)
two_loop_time, out_2_loops = time_function(knn_test.compute_distances_two_loops, X_test)
no_loop_time, out_no_loops = time_function(knn_test.compute_distances_no_loops, X_test)
assert np.log(two_loop_time) - np.log(no_loop_time) > np.log(50)

### Test 5: Labels prediction (0.3 points)

In [ ]:
np.random.seed(42)
for k in [1, 3, 5, 7, 9]:
    predicted_labels = knn_test.predict(X_test, k=k)
    predicted_labels = np.array(predicted_labels, dtype=int).squeeze()
    real_labels = np.array(y_ref_predictions[k], dtype=int).squeeze()
    assert np.array_equal(predicted_labels, real_labels), 'Wrong answer for k={}'.format(k)